In [1]:
""" Copyright:
Jun Hirabayashi (jun@hirax.net) (2018), 
based on Charles McGuinness(charles@mcguinness.us)'s
Simple Focus Stacker(2015)
License:    Apache License 2.0
"""

import numpy as np
import cv2, os

def findHomography(img1kpts, img2kpts, matches):
    img1pts = np.zeros((len(matches), 1, 2), dtype=np.float32)
    img2pts = np.zeros((len(matches), 1, 2), dtype=np.float32)
    for i in range(0,len(matches)):
        img1pts[i] = img1kpts[ matches[i].queryIdx ].pt
        img2pts[i] = img2kpts[ matches[i].trainIdx ].pt
    homography, mask = cv2.findHomography(img1pts, img2pts, 
                       cv2.RANSAC, ransacReprojThreshold=2.0)
    return homography

def alignImages(imgs):
    imgsKPs = []; imgsDescs = []
    imageNum = len(imgs); refImageID = round(imageNum/2)
    detector = cv2.ORB_create(1000)    
    for i in range(0, imageNum-1):
        grayImg = cv2.cvtColor(imgs[i],cv2.COLOR_BGR2GRAY)
        imgKP, imgDesc = detector.detectAndCompute(
            cv2.cvtColor(imgs[i],cv2.COLOR_BGR2GRAY), None)
        imgsKPs.append(imgKP)
        imgsDescs.append(imgDesc)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    homographyMatrix = []
    alignImages = []
    for i in range(0, imageNum-1):
        rawMatches = bf.match(imgsDescs[i], imgsDescs[refImageID])
        sortMatches = sorted(rawMatches, key=lambda x: x.distance)
        matches = sortMatches[0:128]
        homographyMatrix.append( findHomography(
                imgsKPs[i], imgsKPs[refImageID], matches) )
        alignImages.append( cv2.warpPerspective( imgs[i], 
                            homographyMatrix[i], 
                            (imgs[i].shape[1], imgs[i].shape[0]), 
                            flags=cv2.INTER_LINEAR) )
    return alignImages

def focusStack(images):
    alignedImages = alignImages(images)
    gaussianLaplacians = []
    for i in range(len(alignedImages)):
        blurred = cv2.GaussianBlur(cv2.cvtColor(
                images[i],cv2.COLOR_BGR2GRAY), (3, 3), 0)
        gaussianLaplacians.append(cv2.Laplacian(blurred, cv2.CV_64F, ksize=5))
    gaussianLaplacians = np.asarray(gaussianLaplacians)
    aShape = alignedImages[0].shape; aDtype = alignedImages[0].dtype
    output = np.zeros(shape=aShape,dtype=aDtype)
    depth = np.zeros(shape=aShape, dtype=aDtype )
    for y in range(0, alignedImages[0].shape[0]):
        for x in range(0, alignedImages[0].shape[1]):
            gaussianLaplacian = abs(gaussianLaplacians[:, y, x])
            maxIdx = (np.where(gaussianLaplacian == max(gaussianLaplacian)))[0][0]
            output[y,x] = images[maxIdx][y,x]
            depth[y,x] = maxIdx 
    return  (output, depth)

In [ ]:
image_files = sorted(os.listdir( "./" ))
focusimages = []
for img in image_files:
    if img.split(".")[-1].lower() in ["jpg"]:
        focusimages.append( cv2.imread("./{}".format(img)) )
merged, depth = focusStack(focusimages)
cv2.imwrite("merged.png", merged)
cv2.imwrite("depth.png", depth)

In [ ]:
import os
"""
    Author:     Charles McGuinness (charles@mcguinness.us)
    Copyright:  Copyright 2015 Charles McGuinness
    License:    Apache License 2.0
"""
def focusStack(image_files):
    focusimages = []          # 画像ファイルパスのリスト
    for img in image_files:   # から、CV画像のリストを作成する
        print( "Reading in file {}".format(img) )
        focusimages.append( cv2.imread("input_2/{}".format(img)) )
    merged, depth = focus_stack(focusimages) # フォーカススタッキング画像を作成
    cv2.imwrite("merged.png", merged)
    cv2.imwrite("depth.png", depth)

image_files = sorted(os.listdir( "input_2" )) # 画像ファイルパスのリストを作成
for img in image_files:
    if img.split(".")[-1].lower() not in ["jpg", "jpeg", "png"]:
        image_files.remove(img)
focusStack(image_files)